In [1]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from string import punctuation
import torch


emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)
punctuation += '—'
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

d:\lct-yakut-2023\apps\algorithms\LofDT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clear_from_stopwords(text, sw):
    text = re.sub(emoj, '', text)
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2 and i.text not in sw:
            t.append(i.text)
    return ' '.join(t)

In [17]:
check_text = vectorizer(**tokenizer([clear_from_stopwords('с днем рождения\n 😃 Картинки и открытки ♥ vk.com/fotomagic_su ♥', stop_words)], padding=True, truncation=True, pad_to_multiple_of=512, max_length=512, return_tensors='pt'))[0].squeeze(0).sum(dim=0)

In [44]:
def vec_desc(text, n):
    # vec = torch.zeros(1024).squeeze(0)
    # for i in range(len(text.split())//n+1):
    #     vec += vectorizer(**tokenizer([clear_from_stopwords(text[i*n:min((i+1)*n, len(text.split())-1)], stop_words)], padding=True, truncation=True, pad_to_multiple_of=256, max_length=256, return_tensors='pt'))[0].squeeze(0).sum(dim=0)/256
    # return vec
    return vectorizer(**tokenizer([clear_from_stopwords(text, stop_words)], padding=True, truncation=True, pad_to_multiple_of=256, max_length=256, return_tensors='pt'))[0].squeeze(0).sum(dim=0)/256
    

In [45]:
vec_desc(pair_spec['desc'][0], 300)

tensor([ 1.0173,  0.0598, -0.4026,  ...,  0.3758,  0.4787,  0.3521],
       grad_fn=<DivBackward0>)

In [46]:
pair_spec = pd.read_csv('../DATA/spec_num_name_pairs.csv')
pair_spec.head()

,num,name,index,desc
0,35.02.05,Агрономия,0,выращивает ягоды овощи фрукты грибы кормовые т...
1,35.03.03,Агрохимия и агропочвоведение,1,выращивает ягоды овощи фрукты грибы кормовые т...
2,35.03.04,Агрономия,2,выращивает ягоды овощи фрукты грибы кормовые т...
3,35.03.05,Садоводство,3,выращивает ягоды овощи фрукты грибы кормовые т...
4,05.03.01,Геология,4,исследует месторождения полезных ископаемых по...


In [47]:
t = 0
d = dict()
d_l = dict()
for i in pair_spec['desc']:
    d[t] = torch.nn.CosineSimilarity(dim=1)(vec_desc(i, 300), check_text.unsqueeze(0)).item()
    t+=1



In [40]:
d

{0: 0.25295519828796387,
 1: 0.25295519828796387,
 2: 0.25295519828796387,
 3: 0.25295519828796387,
 4: 0.18090833723545074,
 5: 0.21444687247276306,
 6: 0.18090833723545074,
 7: 0.18090833723545074,
 8: 0.27334991097450256,
 9: 0.27334991097450256,
 10: 0.27334991097450256,
 11: 0.27334991097450256,
 12: 0.27334991097450256,
 13: 0.27334991097450256,
 14: 0.27334991097450256,
 15: 0.27334991097450256,
 16: 0.18895220756530762,
 17: 0.20706821978092194,
 18: 0.1823093742132187,
 19: 0.20706821978092194,
 20: 0.1977030336856842,
 21: 0.1977030336856842,
 22: 0.1977030336856842,
 23: 0.1977030336856842,
 24: 0.1977030336856842,
 25: 0.14118324220180511,
 26: 0.14685159921646118,
 27: 0.14118324220180511,
 28: 0.2108883261680603,
 29: 0.2108883261680603,
 30: 0.2108883261680603,
 31: 0.2108883261680603,
 32: 0.2108883261680603,
 33: 0.3477294147014618,
 34: 0.34950903058052063,
 35: 0.3329892158508301,
 36: 0.3823511302471161,
 37: 0.3174680769443512,
 38: 0.35001063346862793,
 39: 0.2614

In [43]:
pd.Series(d).sort_values(ascending=0)

73    0.459578
74    0.459578
76    0.453553
72    0.435658
75    0.435658
        ...   
41    0.148528
26    0.146852
25    0.141183
27    0.141183
50    0.113004
Length: 167, dtype: float64

In [41]:
pair_spec.iloc[pd.Series(d).sort_values(ascending=0).index]

,num,name,index,desc
73,09.02.05,Прикладная информатика (по отраслям),73,создает внешний сайта помощью кода frontend ан...
74,09.02.07,Информационные системы и программирование,74,создает внешний сайта помощью кода frontend ан...
76,09.03.04,Программная инженерия,76,создает внешний сайта помощью кода frontend ан...
72,09.02.03,Программирование в компьютерных системах,72,создает внешний сайта помощью кода frontend ан...
75,54.03.01,Дизайн,75,создает внешний сайта помощью кода frontend ан...
...,...,...,...,...
41,38.03.07,Товароведение,41,продает товары находит новых заключает сделки ...
26,24.05.01,"Проектирование, производство и эксплуатация ра...",26,защищает граждан территории внутренних внешних...
25,23.05.02,Транспортные средства специального назначения,25,защищает граждан территории внутренних внешних...
27,11.03.02,Инфокоммуникационные технологии и системы связи,27,защищает граждан территории внутренних внешних...


In [7]:
data.iloc[pd.Series(d_l).sort_values(ascending=0).index]

,Отрасль текстом,Название профессии
№,,
26,Легкая промышленность,Швея-закройщица
14,Здравоохранение и медицина,Реабилитолог
8,"Бизнес, менеджмент и деловые услуги",Маркетолог
29,Машиностроение и производство транспортных сре...,Сварщик
17,Информационные технологии (ИТ) и телекоммуникации,Архитектор интеллектуальных систем управления ...
41,Торговля,Менеджер по работе с маркетплейсами
45,Транспорт и логистика,Автомеханик
43,Транспорт и логистика,Водитель
22,Информационные технологии (ИТ) и телекоммуникации,Специалист по машинному обучению


# Работает долго, попытаемся ускорить

построим модель